In [1]:
import numpy as np
import pandas as pd
import pre_processing as pp

from matplotlib import pyplot as plt

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

C:\Users\Pasha\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
training = pp.get_training()
test = pp.get_test()

In [3]:
X_train = np.array([x for x, y in training])
y_train = np.array([y for x, y in training])
X_test = test
X_train = X_train.reshape(X_train.shape[0], 1, 64, 64)
X_train = X_train.reshape(-1, 64, 64)
y_train = y_train.reshape(-1, 1)
X_train = X_train.reshape(len(X_train),-1)

In [4]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = cross_validation.train_test_split(X_train/255.,y_train,test_size=0.2,random_state=0)

In [15]:
#K_values = [1,5,10,25,100,500,1000,5000,10000]
K_values = [5,10,25]
params = {'n_neighbors':K_values}

clsf = KNeighborsClassifier()
grid_knn = GridSearchCV(estimator=clsf,param_grid=params,n_jobs=-1, verbose=2)
grid_knn.fit(X_train_cv, y_train_cv)

sorted(grid_knn.cv_results_.keys())

best_classifier = grid_knn.best_estimator_
best_params = grid_knn.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 142.7min remaining: 71.4min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 143.1min finished
C:\Users\Pasha\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [16]:
scores = grid_knn.cv_results_['mean_test_score'].reshape(len(K_values))
#plot_param_space_scores(scores, C_values)
best_clsf = best_classifier.fit(X_train_cv,y_train_cv)

C:\Users\Pasha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
y_pred_knn = best_clsf.predict(X_test_cv)

In [18]:
acc_knn = accuracy_score(y_test_cv, y_pred_knn)
print("K-NearestNeighbors accuracy: ",acc_knn)


K-NearestNeighbors accuracy:  0.911


In [19]:
best_classifier

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [9]:
acc_knn = accuracy_score(y_test_cv, y_pred_knn)
print("K-NearestNeighbors accuracy: ",acc_knn)

K-NearestNeighbors accuracy:  0.8986


In [ ]:
knn_deep_pred = best_clsf.predict(X_test)

In [ ]:
knn_deep_pred_nums = np.array([np.argmax(pred) for pred in knn_deep_pred])
knn_deep_pred_final = np.array([(i, x) for (i, x) in enumerate(cnn_deep_pred_nums)])
np.savetxt('../data/test_y_deep_knn.csv', cnn_deep_pred_final, delimiter=',', fmt='%d', header='Id,Label')